# MobileNetV3 Performances - Before imbalanced Dataset 

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader
import numpy as np
import time
import copy
from collections import Counter

# ------------------------------------------------------------
# DEVICE
# ------------------------------------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# ------------------------------------------------------------
# DATA PATH
# ------------------------------------------------------------
train_dir = "mri_dataset_images/train"
val_dir = "mri_dataset_images/test"

classes = ['glioma_tumor', 'meningioma_tumor', 'no_tumor', 'pituitary_tumor']
print("Classes:", classes)

# ------------------------------------------------------------
# AUGMENTATION (STRONG)
# ------------------------------------------------------------
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.RandomRotation(20),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ColorJitter(brightness=0.25, contrast=0.25),
    transforms.RandomAffine(10),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# ------------------------------------------------------------
# DATASET + LOADER
# ------------------------------------------------------------
train_ds = datasets.ImageFolder(train_dir, transform=train_transform)
val_ds = datasets.ImageFolder(val_dir, transform=val_transform)

train_loader = DataLoader(train_ds, batch_size=16, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=16, shuffle=False)

dataloaders = {"train": train_loader, "val": val_loader}

# ------------------------------------------------------------
# CLASS WEIGHTS (FIX FOR IMBALANCED DATA)
# ------------------------------------------------------------
class_counts = Counter(train_ds.targets)
print("Class counts:", class_counts)

weights = []
for i in range(len(classes)):
    weights.append(1.0 / class_counts[i])

weights = torch.tensor(weights).float().to(device)
criterion = nn.CrossEntropyLoss(weight=weights)

# ------------------------------------------------------------
# MIXUP FUNCTION
# ------------------------------------------------------------
def mixup_data(x, y, alpha=0.4):
    lam = np.random.beta(alpha, alpha) if alpha > 0 else 1
    batch_size = x.size()[0]
    index = torch.randperm(batch_size)

    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam

def mixup_criterion(criterion, pred, y_a, y_b, lam):
    return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)

# ------------------------------------------------------------
# LOAD MobileNetV3-Large + FINE TUNE
# ------------------------------------------------------------
model = models.mobilenet_v3_large(weights=models.MobileNet_V3_Large_Weights.IMAGENET1K_V1)

# Freeze all base layers
for param in model.features.parameters():
    param.requires_grad = False

# Unfreeze last 3 blocks
for param in model.features[-3:].parameters():
    param.requires_grad = True

# Replace classifier (INCREASE DROPOUT)
model.classifier = nn.Sequential(
    nn.Linear(model.classifier[0].in_features, 1024),
    nn.Hardswish(),
    nn.Dropout(0.4),   # increased dropout to reduce overfitting
    nn.Linear(1024, 4)
)

model = model.to(device)

# ------------------------------------------------------------
# OPTIMIZER
# ------------------------------------------------------------
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)

# ------------------------------------------------------------
# TRAINING FUNCTION (WITH TRAIN ACCURACY)
# ------------------------------------------------------------
def train_model(model, criterion, optimizer, dataloaders,
                num_epochs=25, patience=5):

    best_acc = 0.0
    best_wts = copy.deepcopy(model.state_dict())
    no_improve = 0

    for epoch in range(1, num_epochs + 1):
        start = time.time()
        print(f"\nEpoch {epoch}/{num_epochs}")
        print("---------------------------------------")

        for phase in ["train", "val"]:
            if phase == "train":
                model.train()
            else:
                model.eval()

            running_loss = 0
            running_corrects = 0
            total_samples = 0

            for inputs, labels in dataloaders[phase]:
                inputs, labels = inputs.to(device), labels.to(device)
                optimizer.zero_grad()

                # MIXUP only during training
                if phase == "train":
                    inputs, targets_a, targets_b, lam = mixup_data(inputs, labels)
                    outputs = model(inputs)
                    loss = mixup_criterion(criterion, outputs, targets_a, targets_b, lam)
                    _, preds = torch.max(outputs, 1)

                    # Approx accuracy for Mixup
                    running_corrects += (lam * (preds == targets_a).sum().item()
                                        + (1 - lam) * (preds == targets_b).sum().item())
                else:
                    with torch.no_grad():
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)
                        _, preds = torch.max(outputs, 1)
                        running_corrects += (preds == labels).sum().item()

                if phase == "train":
                    loss.backward()
                    optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                total_samples += inputs.size(0)

            epoch_loss = running_loss / total_samples
            epoch_acc = running_corrects / total_samples

            if phase == "train":
                print(f"Train Loss: {epoch_loss:.4f} | Train Acc: {epoch_acc*100:.2f}%")
            else:
                print(f"Val Loss: {epoch_loss:.4f} | Val Acc: {epoch_acc*100:.2f}%")

                # Save best model
                if epoch_acc > best_acc:
                    best_acc = epoch_acc
                    best_wts = copy.deepcopy(model.state_dict())
                    torch.save(best_wts, "best_mobilenetv3_mri.pth")
                    print(" Best model saved!")
                    no_improve = 0
                else:
                    no_improve += 1
                    print(f" No improvement, patience left: {patience - no_improve}")

        print(f" Time: {time.time() - start:.2f} sec")

        if no_improve >= patience:
            print("\n Early Stopping Activated!")
            break

    print("\n Training Completed!")
    print("Best Model Saved As: best_mobilenetv3_mri.pth")

    model.load_state_dict(best_wts)
    return model

# ------------------------------------------------------------
# TRAIN MODEL
# ------------------------------------------------------------
model = train_model(model, criterion, optimizer, dataloaders)


Device: cpu
Classes: ['glioma_tumor', 'meningioma_tumor', 'no_tumor', 'pituitary_tumor']
Class counts: Counter({3: 827, 0: 826, 1: 822, 2: 395})

Epoch 1/25
---------------------------------------
Train Loss: 0.9521 | Train Acc: 62.18%
Val Loss: 1.3179 | Val Acc: 46.45%
 Best model saved!
 Time: 258.06 sec

Epoch 2/25
---------------------------------------
Train Loss: 0.7659 | Train Acc: 71.48%
Val Loss: 1.6726 | Val Acc: 40.10%
 No improvement, patience left: 4
 Time: 273.11 sec

Epoch 3/25
---------------------------------------
Train Loss: 0.6756 | Train Acc: 75.28%
Val Loss: 1.8475 | Val Acc: 41.12%
 No improvement, patience left: 3
 Time: 263.65 sec

Epoch 4/25
---------------------------------------
Train Loss: 0.6461 | Train Acc: 76.64%
Val Loss: 1.1847 | Val Acc: 58.63%
 Best model saved!
 Time: 238.42 sec

Epoch 5/25
---------------------------------------
Train Loss: 0.6276 | Train Acc: 77.92%
Val Loss: 0.8912 | Val Acc: 68.53%
 Best model saved!
 Time: 193.62 sec

Epoch 6/2